# Import

In [ ]:
import pandas as pd

import scipy
import functools

import numpy as np
np.random.seed(0)

from typing import List

import random
import math

from scipy.stats import binom
from scipy.stats import chi2_contingency

from tqdm import tqdm

import plotly.express as px

# Puzzles

## 1


She can choose 1, 2, 3, or 4 econ courses. Each will have their own set of possible permutations. Assuming order doesn't matter, for each quantity E of econ courses $\in {1,2,3,4}$ there will be ${4\choose E}$ ways to pick econ courses, and $4-E\choose S$ many ways to choose the number of science courses. Therefore, for each E $\in {1,2,3,4}$, the total number of possible permutations is ${4\choose E} {4-E\choose S}$. To get the total number of permutations, we add each of these together, i.e., 

Total Permuations = $$ \sum_{E \in {1,2,3,4}} {4\choose E} {4-E\choose S} = 425$$

In [ ]:
n_choose_k = functools.partial(scipy.special.comb, exact=True) #just sets exact=True

In [ ]:
esss = n_choose_k(N=4, k=1) * n_choose_k(N=8, k=3)
eess = n_choose_k(N=4, k=2) * n_choose_k(N=8, k=2)
eees = n_choose_k(N=4, k=3) * n_choose_k(N=8, k=1)
eeee = n_choose_k(N=4, k=4) * n_choose_k(N=8, k=0)

print(f'Total # permutations = {esss + eess + eees + eeee}')

Total # permutations = 425


## 2

Assumptions:  
1. The penny doesn't bounce off the board.
2. The penny bounces independently of where it lands on the board.

The penny has a radius of $\frac{0.75}{2} = 0.375$ inches, and the penny must land entirely within the board, so the center of the penny must land greater than radius=0.375 inches away from the edge of the board. This leaves a square of "safe" landings for the center of the penny; this square has side-lengths of $2.25 - radius = 2.25-0.375 = 1.875$. To calculate the odds of the center of the penny landing within the "safe" square, given the assumptions listed above, we take the area of the smaller "safe" square and divide it by the area of the board. This works out to be $$\approx 0.694$$

In [2]:
r = (.75/2)
inner_area = (2.25-r)**2
board_area = 2.25**2
print(inner_area / board_area)

0.6944444444444444


## 3

Denote all people i in [1-11] as P_i. WLOG, let P_11 be me.  

Assume for now that nobody is shaking my hand. For all i in [1,10], WLOG, say that P_i shakes hands with the i-highest ID'd people in [1,10], e.g., P_2 shakes hands with the 2 highest people: P_10 and P_9. This works for i in [1-5], however, once we cross the mid-point of this range from 5 to 6, there are no longer i-many eligible people higher than i. For example, P_6 shakes hands with the 6 highest IDs in [1-10] (excepting themselves), which are [4, 5, 7, 8, 9, 10]. The issue here is that if P_4 shakes hands with P_6, the "algorithm" is broken, in the sense that P_4 is no longer shaking hands with the 4 highest IDs, since P_4 is now shaking hands with P_6 too! To summarize the problem here, all P_i in [6,10] need to shake one more hand after shaking hands with all eligible people in [1,10]. The solution is for me, P_11, to shake hands with all P_i in [6,10].

This specific solution works out to me (P_11) shaking 5 hands [6, 7, 8, 9, 10], and all P_i in [1,10] shaking i-1 hands. Since the phrasing of the question implies that there is only one valid solution, this arbitrary solution must be unique.

## 4

4 people.

Let $J=$ number of people at CAAI whose names start with J.  
Let $N=$ number of people at CAAI.  

Let $P_1, P_2 = $ the first and second people you meet at CAAI, respectively.  

Then $\Pr\big((P_1 \in J) \land (P_2 \in J)\big) = \frac{1}{2} = \frac{J}{N} * \frac{J-1}{N-1}$

Assuming that $N \leq 50$, the following code indicates that there are two valid possibilities:
* $J=3, N=4$
* $J=15, N=21$

Given the hint, I'd guess that 3 out of 4 people at CAAI have names that start with J. On the other hand, a quick glance at your website shows 11 people in total, and 5 with names that start with "J".

In [ ]:
def is_sol(n:int, j:int) -> bool:
    '''if j, n satisfy probability equality'''
    if j > n:
        return False
    try:
        p = (j / n) * (j - 1) / (n - 1)
    except ZeroDivisionError:
        return False
    return np.isclose(p, 1 / 2)

In [ ]:
def compute_feasible_array(max_n=50) -> pd.DataFrame:
    '''
    create 50X50 array of feasible combinations of j, n 
    where each cell == True iff that col/row combo is valid
    '''
    q3_results = pd.DataFrame(
        [
            [is_sol(n=n, j=j) for n in range(max_n)] 
        for j in range(50)
            ]
    )
    return q3_results

In [ ]:
def fetch_solutions(max_n=50) -> List[tuple]:
    '''
    get feasible solutions from array of feasibility combos
    '''
    # simulate
    q3_results = compute_feasible_array(max_n=max_n)
    
    # whittle down array to rows and cols that contain True
    cols_to_keep = q3_results.sum() >= 1 
    rows_to_keep = q3_results.sum(axis=1) >= 1
    sols = q3_results[rows_to_keep].loc[:, cols_to_keep]

    return sols

fetch_solutions()

,4,21
3,True,False
15,False,True


## 5

No, I don't think a fair, random die was used in either case, although it's more likely that the die was fair with 20 than with 100. Rolls of 1,2,3,5 received approximately their share (1/6) of rolls, but 4 and 6 are large outliers. 

Intuitively, fair die were more likely to have been used in the case of 20 rolls because the law of large numbers hasn't "kicked in" as much, i.e., larger outliers are less suprising, since as $n \to \infty, P(x=i) \to \frac{1}{6}, \,\, \forall i \in \{1,2,3,4,5,6\}$, and 100 is "closer" to infinity than 20 is.

## 6 (Menu-Logging)

$ \mathbb{E}[Visits] \approx 119.8765$

In [ ]:
def choose_a_dish() -> int:
    '''simulate uniform RV over support [0,29]'''
    return np.random.randint(low=0, high=30, size=1)[0]

In [ ]:
def go_to_restaurant_until_all_tried() -> int:
    '''
    Simulate repeatedly visiting the restaurant and choosing a dish accoriding to unif[0,29] dist. 
    Seeing how many times it takes to try all dishes
    '''
    all_tried = False # bool for whether all dishes have been tried or not
    times_visited = 0 # keep track of num. times the restaurant is visited
    dishes_tried = {i:False for i in range(0,30)} # keep track of dishes tried in this dict

    while not all_tried:
        dish = choose_a_dish()
        dishes_tried[dish] = True
        times_visited += 1
        if sum(dishes_tried.values()) == 30: # if all dishes tried
            all_tried = True
            
    return times_visited

In [ ]:
def simulate_n_visits(n=10000, return_runs=False):
    runs = np.array([go_to_restaurant_until_all_tried() for i in tqdm(range(n))])
    
    expectation = runs.mean()
    
    print(f'Expected # Visits = {expectation}')
    
    if return_runs:
        return runs
    

runs = simulate_n_visits(250000, return_runs=True)


# plot (just to see). looks vageuly Poisson-ish or log-normal
fig = px.histogram(x=runs, histnorm='probability density')
fig.add_vline(x=runs.mean(), line_width=2, line_dash="dash", line_color="black")
fig.show()

 37%|███▋      | 372214/1000000 [19:21<32:38, 320.57it/s]


Expected # Visits = 119.8765
